In [1]:
from class_definitions import *

Wines[:5]

[Wine: Zweigelt 2016 Bauer, Source: name-search,
 Wine: Truffle Hound 2017 Unico Zelo, Source: name-search,
 Wine: Sforzato di Valtellina 2013 Alfio Mozzi, Source: name-search,
 Wine: Cabernet Sauvignon 2017 Oracle Cellars, Source: name-search,
 Wine: Spatburgunder Kabinett 2017 Koehler-Ruprecht, Source: name-search]

In [2]:
from vivino_db import database

Engine(sqlite://///Users/schlinkertc/code/wine_project/wine/vivino_db/vivino_wines.db)


In [3]:
db = database.db

In [4]:
vintages_df = pd.read_sql_table('vintages',db.db_engine)
wines_df = pd.read_sql('select * from wines',db.db_engine)
styles_df = pd.read_sql_table("styles",db.db_engine)
regions_df = pd.read_sql_table("regions",db.db_engine)
wineries_df = pd.read_sql_table("wineries",db.db_engine)

In [5]:
db.db_engine.table_names()

['regions', 'styles', 'vintages', 'wineries', 'wines']

### Fuzzy matching from KCW to vivino

In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
vivino_vintages = vintages_df['name'].to_list()

In [9]:
stmt = """
SELECT 
    v.name as wine,
    ws.name as winery
FROM 
    vintages v
INNER JOIN 
    wines w on v.wine_id==w.id
INNER JOIN 
    wineries ws on w.winery_id=ws.id
"""

In [10]:
pd.read_sql(stmt,db.db_engine)

,wine,winery
0,Korta Katarina Plavac Mali,Korta Katarina
1,West+Wilder White,West+Wilder
2,Ostatu Rioja Tinto,Ostatu
3,Le Pianate Un Paio Rosso,Le Pianate
4,Dirupi Vino Sbagliato Sforzato di Valtellina,Dirupi
...,...,...
1462,C&H Fournaise Blanc de Blancs Champagne,C&H Fournaise
1463,Contini Attilio Brut,Contini
1464,Domaine du Bel Air Les Marsaules Bourgueil,Domaine du Bel Air
1465,Casillero del Diablo Sauvignon Blanc (Reserva),Casillero del Diablo


In [20]:
wine_matches = []
problems = []
for wine in Wines:
    search_string = "".join([x+' ' for x in wine.search_terms]).strip(' ')
    matches = [(x,fuzz.token_sort_ratio(search_string,x),wine) for x in vivino_vintages]
    best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
    if best_match[1] > 60:
        wine_matches.append(best_match)
    else:
        matches = [(x,fuzz.token_sort_ratio(wine.name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        if best_match[1] > 60:
            wine_matches.append(best_match)
        else:
            problems.append(best_match)

In [26]:
wine_matches = []
for wine in Wines:
    if len(wine.wineSearcher.keys())>2:
        wine_name = wine.wineSearcher['name']
        matches = [(x,fuzz.token_sort_ratio(wine_name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        wine_matches.append(best_match)
    else:
        matches = [(x,fuzz.token_sort_ratio(wine.name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        if best_match[1] > 60:
            wine_matches.append(best_match)
        else:
            search_string = "".join([x+' ' for x in wine.search_terms]).strip(' ')
            matches = [(x,fuzz.token_sort_ratio(search_string,x),wine) for x in vivino_vintages]
            best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
            wine_matches.append(best_match)


In [30]:
problems = list(filter(lambda x: x[1]<70, wine_matches))

In [31]:
len(problems)

66

In [32]:
problems

[('Proprieta Sperino Uvaggio',
  66,
  Wine: Costa delle Sesia Uvaggio 2015 Sperino, Source: name-search),
 ('Lammershoek LBV Cape Vintage Zinfandel',
  60,
  Wine: Zinfandel 2017 The People's Wine Co., Source: name-search),
 ('Domaine Saint Germain La Cuvée des Baujues Rouge',
  65,
  Wine: Jacquère Cuvée Baujues 2016 St. Germain, Source: name-search),
 ('Xanthos Fly By Chardonnay',
  65,
  Wine: Chardonnay 2017 Nathan K., Source: name-search),
 ('Risky Grapes La Traca Bobal',
  61,
  Wine: Bobal 2018 La Traca, Source: name-search),
 ('Corte Manzini Bolla Rossa',
  65,
  Wine: Lambrusco Bollarossa NV Corte Manzini, Source: name-search),
 ('Torremorón Roble',
  61,
  Wine: Ribera del Duero 2018 Torremorón, Source: name-search),
 ('A Blooming Hill Mingle White Blend',
  61,
  Wine: White Blend 2019 Bloem, Source: name-search),
 ("Merabs's Wine Cellar Imereti Krakhuna",
  65,
  Wine: Imereti 2018 Amiran's Wine Cellar, Source: name-search),
 ('Sanfeletto Conegliano-Valdobbiadene Prosecco 

In [11]:
problem_ids = []
for problem in problems:
    problem_ids.extend([vw['wine_link'].split('/')[-1] for vw in problem[-1].vivino])

In [12]:
len(problem_ids)

228

In [52]:
from vivino_scraper import *

In [55]:
import time
still_not_added = []
for wine_id in ["26696058"]:
    try:
        result = scrape_vivino(wine_id)
        joblib.dump(result,storage_directory+str(wine_id))
        print('*')
        time.sleep(5)
    except:
        still_not_added.append(wine_id)
        continue

*


### Bringing it all together

In [18]:
vintages_df.iloc[vintages_df[vintages_df['name']==least_confident].index[0]]

index                                                              520
id                                                           159955768
seo_name                              gassier-elevae-rose-d-elevage-uv
name                                     Gassier Elevae Rosé d'Elevage
statistics_status                                               Normal
statistics_ratings_count                                            10
statistics_ratings_average                                         4.4
statistics_labels_count                                              4
statistics_reviews_count                                             0
wine_id                                                        7134802
description                                                           
ratings_distribution_1                                               0
ratings_distribution_2                                               0
ratings_distribution_3                                               1
rating

In [41]:
wines_df.iloc[wines_df[wines_df['id']==2237206].index[0]]

index                                           29
id                                         2237206
name                                      Zweigelt
seo_name                                  zweigelt
type_id                                          1
is_natural                                       0
region_id                                      869
review_status                                    2
winery_id                                    42667
statistics_status                           Normal
statistics_ratings_count                        26
statistics_ratings_average                     3.5
statistics_labels_count                        112
statistics_vintages_count                       12
hidden                                           0
description                                       
non_vintage                                      0
alcohol                                         13
created_at                    2014-03-09T08:55:09Z
is_first_wine                  

In [15]:
styles_df.iloc[styles_df[styles_df['id']==255].index[0]]

index                                                                        8
id                                                                         255
seo_name                                                     austrian-zweigelt
regional_name                                                         Austrian
varietal_name                                                         Zweigelt
name                                                         Austrian Zweigelt
body                                                                         4
body_description                                                   Full-bodied
acidity                                                                      3
acidity_description                                                       High
country_code                                                                at
country_name                                                           Austria
country_native_name                                 

In [16]:
regions_df.iloc[regions_df[regions_df['id']==869].index[0]]

index                                28
id                                  869
name                             Wagram
name_en                                
seo_name                      donauland
country_code                         at
country_name                    Austria
country_native_name          Österreich
country_seo_name                austria
country_regions_count                24
country_users_count              147508
country_wines_count               43242
country_wineries_count             4240
background_image_location          None
Name: 28, dtype: object

In [17]:
wineries_df[wineries_df['id']==42667]

,index,id,name,seo_name,status,review_status,statistics_ratings_count,statistics_ratings_average,statistics_labels_count,statistics_wines_count
28,28,42667,Stefan Bauer,stefan-bauer,0,Completed,355,3.9,1264,17


In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [19]:
import time
def selenium_scrape(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    
    chrome_driver_path = "/Users/schlinkertc/code/chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 1200)")
    driver.execute_script("window.scrollTo(0, 1800)")
    time.sleep(2)
    master_wine_page = driver.find_element_by_id('master-wine-page-app')
    
    driver.set_window_size(width=742,height=983)
    a_tags = driver.find_elements_by_tag_name('a')
    
    for a in a_tags:
        a_class = a.get_attribute('class')
        if 'toggleShowAll' in a_class:
            show_all = a
            break

    a.click()
#     text = master_wine_page.text
#     notes = [x for x in text.split('\n') if 'mentions of' in x]
    
    svgs = master_wine_page.find_elements_by_tag_name('svg')

    tasteNotes = []
    for s in svgs:
        try:
            svg_class = s.get_attribute('class')
            if 'tasteNote' in svg_class:
                tasteNotes.append(s)
        except:
            continue 

    notes = []
    note_tags = []
    for i in range(len(tasteNotes)):
        tasteNotes[i].click()
        time.sleep(5)
        spans = driver.find_elements_by_tag_name('span')
        for span in spans:
            try:
                span_class = span.get_attribute('class')
                if 'tasteReviews__capitalizedTastes' in span_class:
                    note_texts.append(span.text)
            except:
                continue
        divs = driver.find_elements_by_tag_name('div')
        
        
        for div in divs:
            try:
                div_class = div.get_attribute('class')
                if 'noteTag__name' in div_class:
                    note_tags.append({'index':i,'tag':div.text})
            except:
                continue
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        
      ## This is to get the indicator bars. not ready yet  
#     spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]
#     styles = []
#     for span in spans:
#         try:
#             style = span.get_attribute('style')
#             if style.startswith('width'):
#                 styles.append(style)
#         except:
#             continue
    driver.quit()
    indexed_notes = [{'index':n,'note':note} for n,note in enumerate(notes)]
    
    for tag in note_tags:
        for note in indexed_notes:
            if note['index']==tag['index']:
                tag['note']=note['note']
                
    return {wine_id:note_tags}

In [20]:
selenium_scrape(4470482)

{4470482: [{'index': 0, 'tag': 'plum'},
  {'index': 0, 'tag': 'blackcurrant'},
  {'index': 1, 'tag': 'prune'},
  {'index': 2, 'tag': 'violet'},
  {'index': 3, 'tag': 'chard'},
  {'index': 3, 'tag': 'red beet'},
  {'index': 4, 'tag': 'chocolate'}]}

In [165]:
def scrape_wineSearcher_fromVivino(seo_name):
    seo_name = seo_name.strip("-uv").replace('-','+')
    url = f'https://www.wine-searcher.com/find/{seo_name}'

    wine_dict = {'url':url}

    page = requests.get(url,headers=Wine.headers)
    if page.status_code != 200:
        return page.status_code
    bs = BeautifulSoup(page.content,'lxml')

    wine_dict['name'] = bs.find(
        'h1',attrs={'class':'wine'}).\
            getText(strip=True)

    wine_info_panel = bs.find('div',attrs={"class":'wine-info-panel'})
    if wine_info_panel==None:
        return wine_dict
    wine_info = []

    for div in wine_info_panel.find_all('div',attrs={'class':'dtc'}):
        wine_info.append(
            [x.getText() for x in div.find_all(
                'span',attrs={'class':"dtlbl sidepanel-text"}
            ) 
             if type(x)==bs4.element.Tag])

    wine_info = [
        item.split('\n') for sublist in wine_info for item in sublist
    ]
    wine_info = [
        item.strip() for sublist in wine_info 
        for item in sublist if item not in  ['',' ']
    ]

    wine_dict_keys = [
        'Producer','Region/Appellation',
        'Grape/Blend','Food Suggestion',
        'Wine Style','Alcohol Content','Notes'
    ]

    for key in wine_dict_keys:
        try:
            key_index = wine_info.index(key)
            wine_dict[key]=wine_info[key_index+1]
        except:
            wine_dict[key]='null'

    return wine_dict

In [166]:
scrape_wineSearcher_fromVivino("stefan-bauer-zweigelt-uv")

{'url': 'https://www.wine-searcher.com/find/stefan+bauer+zweigelt',
 'name': 'Stefan Bauer Zweigelt, Wagram, Austria',
 'Producer': 'Stefan Bauer',
 'Region/Appellation': 'Wagram',
 'Grape/Blend': 'Zweigelt',
 'Food Suggestion': 'Pork',
 'Wine Style': 'Red - Savory and Classic',
 'Alcohol Content': 'null',
 'Notes': 'null'}

In [ ]:
def compile_data(wine):
    